In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
from nilearn.plotting import plot_img, plot_stat_map, view_img, plot_prob_atlas
from nilearn.regions import connected_label_regions
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from nilearn.image import concat_imgs, mean_img, index_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn.plotting import plot_roi
from nilearn.maskers import NiftiMapsMasker, NiftiSpheresMasker
from scipy.interpolate import interp1d
import seaborn as sns

# Homemade functions


# Check files of sub-01


In [2]:
# Open a datasets directory. 
fd = os.open("/Users/luisalvarez/Documents/Datasets", os.O_RDONLY)

# Use os.fchdir() method to change the current dir/folder.
os.fchdir(fd)

# Safe check- Print current working directory
print("Current working dir : %s" % os.getcwd())

Current working dir : /Users/luisalvarez/Documents/Datasets


In [5]:

# Load files for sub 001  

#sub01_run1_func_file = "MID_preproc/sub-01/func/sub-01_task-MID_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
#sub01_run1_mask_file = "MID_preproc/sub-01/func/sub-01_task-MID_run-1_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
#sub01_file = 'MID_raw/sub-01/func/sub-01_task-MID_run-1_events.tsv'
#sub01_run1_confounds_file = "MID_preproc/sub-01/func/sub-01_task-MID_run-1_desc-confounds_timeseries.tsv"
sub01_events_r1_file = "/Users/luisalvarez/Documents/SpanLab/fmrimatch_psypy/data/ab231203_fmritrailer_run2_2023_Dec_03_1751.csv"
sub01_events_r2_file = "/Users/luisalvarez/Documents/SpanLab/fmrimatch_psypy/data/ab231203_fmritrailer_run2_2023_Dec_03_1806.csv"



Notes on csv files
- Note, we can determine the run of each file by looking into the 'expName' column -> ['fmritrailer_run2', 'fmritrailer_run1']


In [22]:
# Load participants file and get number of trials.
 
sub01_events_r1_df = pd.read_csv(sub01_events_r1_file, sep='\,')
sub01_events_r1_keys = sub01_events_r1_df.keys()

# Determine to which run this file corresponds. 
sub01_events_r1_run = sub01_events_r1_df["expName"][0]
#trials_num = run1_events_raw.shape[0]

print(f"sub01_events_r1_df shape: {sub01_events_r1_df.shape}")
print(f"sub01_events_r1_df keys: {sub01_events_r1_keys}")
print(f"sub01_events_r1_df actual run number: {sub01_events_r1_run}")

#print(f"Run 1 number of trials: {trials_num}")

sub01_events_r1_df.head(40)

fmritrailer_run2
sub01_events_r1_df shape: (22, 120)
sub01_events_r1_df keys: Index(['order', 'vid_path', 'aud_path', 'genre', 'released', 'trial_ITI',
       'scale_flip', 'label', 'Rating_trials.thisRepN',
       'Rating_trials.thisTrialN',
       ...
       'key_resp_end.keys', 'key_resp_end.started', 'key_resp_end.stopped',
       'participant', 'session', 'date', 'expName', 'psychopyVersion',
       'frameRate', 'Unnamed: 119'],
      dtype='object', length=120)


/var/folders/0z/1yt2h6410kb7_mgghf4q28z00000gn/T/ipykernel_9967/2216648285.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sub01_events_r1_df = pd.read_csv(sub01_events_r1_file, sep='\,')


,order,vid_path,aud_path,genre,released,trial_ITI,scale_flip,label,Rating_trials.thisRepN,Rating_trials.thisTrialN,...,key_resp_end.keys,key_resp_end.started,key_resp_end.stopped,participant,session,date,expName,psychopyVersion,frameRate,Unnamed: 119
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
5,1.0,videos/video_mute/rh6_trailer.mp4,videos/audio/rh6_audio.wav,h,1.0,6.0,0.0,rh6,0.0,0.0,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
6,2.0,videos/video_mute/uh1_trailer.mp4,videos/audio/uh1_audio.wav,h,0.0,6.0,1.0,uh1,0.0,1.0,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
7,3.0,videos/video_mute/rc2_trailer.mp4,videos/audio/rc2_audio.wav,c,1.0,6.0,1.0,rc2,0.0,2.0,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
8,4.0,videos/video_mute/rh2_trailer.mp4,videos/audio/rh2_audio.wav,h,1.0,2.0,0.0,rh2,0.0,3.0,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN
9,5.0,videos/video_mute/rh1_trailer.mp4,videos/audio/rh1_audio.wav,h,1.0,2.0,1.0,rh1,0.0,4.0,...,NaN,NaN,NaN,ab231203,1,2023_Dec_03_1751,fmritrailer_run2,2021.2.3,60.242847,NaN


In [21]:
# Select rows with relevant information -> Find row start. 
sub01_events_r1_start = np.where(sub01_events_r1_df['order'] == 1)
print(sub01_events_r1_start)

(array([5]),)


In [14]:
sub01_events_r1_keys[0:50]

Index(['order', 'vid_path', 'aud_path', 'genre', 'released', 'trial_ITI',
       'scale_flip', 'label', 'Rating_trials.thisRepN',
       'Rating_trials.thisTrialN', 'Rating_trials.thisN',
       'Rating_trials.thisIndex', 'text_intro.started', 'text_intro.stopped',
       'key_resp_intro.keys', 'key_resp_intro.rt', 'key_resp_intro.started',
       'key_resp_intro.stopped', 'text_instr_WATCH_Q.started',
       'text_instr_WATCH_Q.stopped', 'key_instr_WATCH_Q.keys',
       'key_instr_WATCH_Q.rt', 'key_instr_WATCH_Q.started',
       'key_instr_WATCH_Q.stopped', 'instr_WATCHscale.response',
       'instr_WATCHscale.rt', 'instr_WATCHscale.started',
       'instr_WATCHscale.stopped', 'text_instr_FEEL_Q.started',
       'text_instr_FEEL_Q.stopped', 'key_resp_FEEL_Q.keys',
       'key_resp_FEEL_Q.rt', 'key_resp_FEEL_Q.started',
       'key_resp_FEEL_Q.stopped', 'instr_FEELscale.response',
       'instr_FEELscale.rt', 'instr_FEELscale.started',
       'instr_FEELscale.stopped', 'text_instr_AROU